In [1]:
import warnings
warnings.filterwarnings("ignore")

import re
import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import torch

from DataLoader.DataLoader import DataLoader
from DataLoader.DataBasedAgent import DataBasedAgent
from DataLoader.DataRLAgent import DataRLAgent
import DeepRLAgent.VanillaInput.Train as Train
from PatternDetectionInCandleStick.Evaluation import Evaluation
import distinctipy
import plotly.graph_objs as go
from kaleido.scopes.plotly import PlotlyScope

from importlib import reload
import shutil

Train = reload(Train)
DeepRL = Train.Train
from utils_best_arm import add_train_portfo, add_test_portfo, plot_return, calc_return, plot_action_point, setup_logger
pd.options.display.max_colwidth = 100

device = "cpu"
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
CURRENT_PATH = os.getcwd()
plt.rcParams["font.size"] = 14

In [2]:
# # NOTE: 把profit_5的pkl文件拿过来
# symbols = os.listdir("./Data/")[:10]
# models = ["model_profit_0", "model_regularized_0.01", "model_regularized_0.05", "model_regularized_0.1", "model_regularized_0.2"]
# for symbol in symbols:
#     for seed in range(100):
#         if not os.path.exists(f"./Results/{symbol}/"):
#             os.mkdir(f"./Results/{symbol}")
#             os.mkdir(f"./Results/{symbol}/2016-01-01~2019-01-01")
#         if not os.path.exists(f"./Results/{symbol}/2016-01-01~2019-01-01/{seed}"):
#             os.mkdir(f"./Results/{symbol}/2016-01-01~2019-01-01/{seed}")
#             os.mkdir(f"./Results/{symbol}/2016-01-01~2019-01-01/{seed}/train")

#         for model in models:
#             src = f"../try7_with_regularizezd_rewards_exp3/Results/{symbol}/2016-01-01~2019-01-01/{seed}/train/{model}_{seed}.pkl"
#             dst = f"./Results/{symbol}/2016-01-01~2019-01-01/{seed}/train/{model}_{seed}.pkl"
#             shutil.copy(dst=dst, src=src)

In [3]:

def plot_action_point(dataTest_agent, data_loader, symbol, name, begin=0, end=120):

    data_test = dataTest_agent
    df1 = data_loader.data_test_with_date[begin:end]

    actionlist = list(data_test.data[data_test.action_name][begin:end])
    df1[data_test.action_name] = actionlist

    buy = df1.copy()
    sell = df1.copy()
    none = df1.copy()

    # NOTE 应该画在close处比较合适吧
    buy['action'] = [c if a == 'buy' else None for a, o, c in zip(df1[data_test.action_name], df1.open, df1.close)]
    sell['action'] = [c if a == 'sell' else None for a, o, c in zip(df1[data_test.action_name], df1.open, df1.close)]
    none['action'] = [c if a == 'None' else None for a, o, c in zip(df1[data_test.action_name], df1.open, df1.close)]

    data=[
        go.Candlestick(x=df1.index, open=df1['open'], high=df1['high'], low=df1['low'], close=df1['close'], increasing_line_color= 'lightgreen', decreasing_line_color= '#ff6961'),
        go.Scatter(x=df1.index, y=buy.action, mode = 'markers', marker=dict(color='red', colorscale='Viridis'), name="buy"), 
        go.Scatter(x=df1.index, y=none.action, mode = 'markers', marker=dict(color='blue', colorscale='Viridis'), name="hold"), 
        go.Scatter(x=df1.index, y=sell.action, mode = 'markers', marker=dict(color='green', colorscale='Viridis'), name="sell")
    ]

    layout = go.Layout(
        autosize = False,
        width = 1000,
        height = 400,
        margin=dict(l=20, r=20, t=20, b=20),
        xaxis_rangeslider_visible=False,
        xaxis_title = "date",
        yaxis_title = "adj close",
    )
    
    figSignal = go.Figure(data=data, layout=layout)
    # figSignal.show()
    scope = PlotlyScope()

    with open(f"./ActionsPlot/{symbol}-{name}.png", "wb") as f:
        f.write(scope.transform(figSignal, format="png", scale=2))


In [4]:
def train(
    DATASET_NAME, 
    split_point='2018-01-01', 
    begin_date='2010-01-01', 
    end_date='2020-08-24', 
    model_start_date="",
    model_end_date="",
    initial_investment=1000,
    transaction_cost=0.0001,
    load_from_file=True,
    reward_type="profit",
    seed=42, 
    state_mode=1,
    n_episodes=5,
    lamb=0.0001,
    GAMMA=0.7, 
    n_step=5, 
    BATCH_SIZE=10, 
    ReplayMemorySize=20,
    TARGET_UPDATE=5,
    window_size=None, 
    train_portfolios={},
    test_portfolios={},
    arms=[],
    show_all = False,
    ratio_threshold=0.9,
):
    data_loader = DataLoader(DATASET_NAME, split_point=split_point, begin_date=begin_date, end_date=end_date, load_from_file=load_from_file)
    
    dataTrain_agent = DataRLAgent(data_loader.data_train, state_mode, 'action_encoder_decoder', device, GAMMA, n_step, BATCH_SIZE, window_size, transaction_cost)
    dataTest_agent = DataRLAgent(data_loader.data_test, state_mode, 'action_encoder_decoder', device, GAMMA, n_step, BATCH_SIZE, window_size, transaction_cost)
    
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)

    agent = DeepRL(data_loader, dataTrain_agent, dataTest_agent, 
                DATASET_NAME,  state_mode, window_size, transaction_cost,
                BATCH_SIZE=BATCH_SIZE, GAMMA=GAMMA, ReplayMemorySize=ReplayMemorySize,
                TARGET_UPDATE=TARGET_UPDATE, n_step=n_step, arms=arms)
    
    path = f"./Results/{DATASET_NAME}/{model_start_date}~{model_end_date}/{seed}/train"
    arm = arms[0]
    name = f'{arm["name"]}_{arm["lamb"]}'
    model_path = f"{path}/model_{name}_{seed}.pkl"
    agent_test = agent.test(initial_investment=initial_investment, test_type='test', model_path=model_path)
    test_portfolio = agent_test.get_daily_portfolio_value()
    test_portfolio = pd.Series(test_portfolio).pct_change(1).fillna(0).values.tolist() 
    model_name = f'DQN-stock:{DATASET_NAME}-reward:{name}-seed:{seed}'
    # plot_action_point(dataTest_agent, data_loader, DATASET_NAME, name, begin=0, end=120)
    return data_loader, {"name": model_name, "portfo": test_portfolio}

In [48]:
initial_investment = 1000


kwargs = {
    "load_from_file": True, 
    "transaction_cost": 0.01,
    "initial_investment": initial_investment,
    "state_mode": 1,
    "GAMMA": 0.7, 
    "n_step": 5, 
    "BATCH_SIZE": 10, 
    "ReplayMemorySize": 20,
    "TARGET_UPDATE": 5,
    "window_size": None, 
    "lamb": 0.0,
}

_file = "AAPL"

_begin_date = '20{}-01-01'
_end_date = '20{}-01-01'
_split_point = '20{}-01-01' 

arms = [
    # { "name": "old_profit", "lamb": 0},
    { "name": "profit", "lamb": 0},
    { "name": "regularized", "lamb": 0.01},
    { "name": "regularized", "lamb": 0.05},
    { "name": "regularized", "lamb": 0.1},
    { "name": "regularized", "lamb": 0.2},
    # { "name": "sharpe", "lamb": 0.01 },
    # { "name": "volatility", "lamb": 10 },
]

dates = [
    ("2019-06-23", "2020-06-22", "2016-01-01", "2019-01-01"),
    # ("2020-06-22", "2021-06-22", "2017-01-01", "2020-01-01"),
    # ("2021-06-23", "2022-06-23", "2018-01-01", "2021-01-01"),
]

random_seeds = 100
results = []
portfolios_saved = {}
files = sorted(os.listdir("./Data/"))
for _file in files[6:7]:
    print(_file)
    portfolios_saved[_file] = {}
    for idx, arm in enumerate(arms[:]):
        results2 = []
        tmp_result = []
        portfolios_saved[_file][f"{arm['name']}-{arm['lamb']}"] = []
        
        for seed in tqdm(range(random_seeds)):
            
            ls = []
            bhs = []
        
            train_portfolios = {}
            test_portfolios = {}

            kwargs.update({
                "begin_date": "2016-01-01", 
                "end_date": "2019-01-01", 
                "split_point": "2018-01-01",
                "model_start_date": "2016-01-01",
                "model_end_date": "2019-01-01",
                "DATASET_NAME": _file,
                "reward_type": "",
                "seed": seed,
                "n_episodes": 140,
                "arms": [arm],
                "show_all": True,
                "ratio_threshold": 3,
                "train_portfolios": train_portfolios,
                "test_portfolios": test_portfolios,
            })

            data_loader, model = train(**kwargs)
            ls.extend(model["portfo"])
            bh = data_loader.data_test_with_date["close"]
            bhs.append(bh)
            
            add_test_portfo(test_portfolios, seed, ls)
            if seed == 0: 
                bhs = pd.concat(bhs, axis=0)
                bh_percentage = bhs.pct_change(1).fillna(0).values
                add_test_portfo(test_portfolios, 'B&H', bh_percentage)
                tmp_result.append(bh_percentage.tolist())
            indexes = calc_return(bh_percentage, test_portfolios)
            results2.append(indexes)
            tmp_result.append(model["portfo"])
        
        portfolios_saved[_file][f"{arm['name']}-{arm['lamb']}"] = tmp_result

        results2_df = pd.concat(results2, axis=1)
        results2_bh = results2_df["B&H"]
        del results2_df["B&H"]
        final = pd.concat([
            results2_bh,
            results2_df.median(axis=1)
        ], axis=1)
        final.columns = [f"{_file}-B&H", f"{_file}-{arm['name']}-{arm['lamb']}"]
        if idx > 0:
            del final[f"{_file}-B&H"]
        results.append(final)


CSCO


100%|██████████| 100/100 [00:30<00:00,  3.28it/s]


# NOTE 计算各类指标

In [49]:
results_df = pd.concat(results, axis=1)
ls = []
results_df = pd.concat(results, axis=1)
symbols = list(set(map(lambda x: x.split("-")[0], results_df.columns)))

for symbol in symbols:
    results_df = results_df.rename({
        f"{symbol}-profit-0": f"{symbol}-FP5",
        f"{symbol}-regularized-0.01": f"{symbol}-FPR-0.01",
        f"{symbol}-regularized-0.05": f"{symbol}-FPR-0.05",
        f"{symbol}-regularized-0.1": f"{symbol}-FPR-0.1",
        f"{symbol}-regularized-0.2": f"{symbol}-FPR-0.2",
    }, axis=1)
    cols = [col for col in results_df.columns if symbol in col]
    tmp = results_df[cols]
    ls.append(tmp)


results_df = pd.concat(ls, axis=1)
results_df.to_csv(f"./ts-run-results/[exp2]returns的各类指标-cost-0.01.csv")
results_df

,CSCO-B&H,CSCO-FP5,CSCO-FPR-0.01,CSCO-FPR-0.05,CSCO-FPR-0.1,CSCO-FPR-0.2
sortino_test,0.925596,-0.511959,-0.545901,-0.325342,-0.277700,-0.421933
sharpe_test,0.660608,-0.351310,-0.373222,-0.241340,-0.206509,-1.412142
risk_test,0.263742,0.256809,0.252187,0.230431,0.204047,0.057903
mdd_test,0.174759,0.306838,0.300255,0.222327,0.180018,0.093228
downrisk_test,0.188235,0.171400,0.169793,0.163710,0.151602,0.134621
cumreturn_test,0.148917,-0.112943,-0.116879,-0.074094,-0.053846,-0.091721


In [50]:
cols = list(filter(lambda x: "CSCO" in x, results_df.columns))
results_df[cols]

,CSCO-B&H,CSCO-FP5,CSCO-FPR-0.01,CSCO-FPR-0.05,CSCO-FPR-0.1,CSCO-FPR-0.2
sortino_test,0.925596,-0.511959,-0.545901,-0.325342,-0.277700,-0.421933
sharpe_test,0.660608,-0.351310,-0.373222,-0.241340,-0.206509,-1.412142
risk_test,0.263742,0.256809,0.252187,0.230431,0.204047,0.057903
mdd_test,0.174759,0.306838,0.300255,0.222327,0.180018,0.093228
downrisk_test,0.188235,0.171400,0.169793,0.163710,0.151602,0.134621
cumreturn_test,0.148917,-0.112943,-0.116879,-0.074094,-0.053846,-0.091721


# NOTE 转latex table

In [59]:
def find(symbol):
    cols = [col for col in results_df.columns if symbol in col]
    tmp = results_df[cols]
    cols = ["-".join(col.split("-")[1:]) for col in list(tmp.columns)]
    tmp.columns = cols
    tmp = tmp.T
    cols = [col.split("_")[0] for col in list(tmp.columns)]
    tmp.columns = cols
    return tmp

In [69]:
import pandas as pd

files = [
    "[exp2-CSCO]returns的各类指标-cost-0.0000.csv", 
    "[exp2-CSCO]returns的各类指标-cost-0.0001.csv", 
    "[exp2-CSCO]returns的各类指标-cost-0.0005.csv", 
    "[exp2-CSCO]returns的各类指标-cost-0.001.csv",
    "[exp2-CSCO]returns的各类指标-cost-0.005.csv",
    "[exp2-CSCO]returns的各类指标-cost-0.01.csv",
]
ls = []
for _file in files:
    results_df = pd.read_csv(f"./ts-run-results/{_file}", index_col=[0])
    cols = list(results_df.columns)
    symbols = []
    for col in cols:
        symbols.append(col.split("-")[0])

    symbols = sorted(list(set(symbols)))
    if _file.endswith("0000.csv"):
        df = results_df.copy()
    else:
        df = find("CSCO")
    df["cost"] = re.findall("-(\d+.\d+).csv", _file)[0]
    df = df.reset_index()
    df = df.rename({
        "index": "criteria",
    }, axis=1)
    ls.append(df)


concat = pd.concat(ls)
concat = concat.set_index(["cost", "criteria"])
concat.to_csv("./ts-run-results/[exp2-CSCO]returns的各类指标-cost-汇总.csv")
concat.head()

sortino    sharpe      risk       mdd  downrisk  cumreturn
cost   criteria                                                             
0.0000 B&H       0.925596  0.660608  0.263742  0.174759  0.188235   0.148917
       FP5       2.031148  1.467095  0.244096  0.178892  0.176289   0.388237
       FPR-0.01  2.323558  1.682880  0.239992  0.178892  0.173190   0.452027
       FPR-0.05  3.112911  2.205786  0.217359  0.131325  0.153171   0.572123
       FPR-0.1   4.574572  3.042705  0.192864  0.103816  0.125362   0.744638

In [58]:
find("CSCO").to_csv("[exp2-CSCO]returns的各类指标-cost-0.0001.csv")

,sortino,sharpe,risk,mdd,downrisk,cumreturn
B&H,0.925596,0.660608,0.263742,0.174759,0.188235,0.148917
FP5,2.002347,1.444751,0.244155,0.179385,0.176108,0.380760
FPR-0.01,2.291477,1.658316,0.240020,0.179385,0.173149,0.446330
FPR-0.05,3.083661,2.180672,0.217392,0.131673,0.153106,0.562874
FPR-0.1,4.521199,3.006375,0.192852,0.103995,0.125418,0.732380
FPR-0.2,0.000000,0.000000,0.047948,0.044120,0.044712,0.000000


In [46]:
def convert_latex(symbol, _df):
    template = "\\begin{table}[H]\n" + \
        "    \\label{table:" + symbol + "}\n" + \
        "    \\caption{" + symbol + "}\n" + \
        "    \\begin{tabular}{l c c c c c c}\n" + \
        "    \\hline\n" + \
        "                     & sortino & sharpe &	risk &  mdd    &  downrisk &	cumreturn     \\\ \n" + \
        "    \\hline\n" + \
        f"    B\\&H	          &  {_df['sortino'].iloc[0]:.4f} & {_df['sharpe'].iloc[0]:.4f}	& {_df['risk'].iloc[0]:.4f} &  {_df['mdd'].iloc[0]:.4f} &  {_df['downrisk'].iloc[0]:.4f}   & 	{_df['cumreturn'].iloc[0]:.4f}        \\\ \n" + \
        f"    FP5	          &  {_df['sortino'].iloc[1]:.4f} & {_df['sharpe'].iloc[1]:.4f}	& {_df['risk'].iloc[1]:.4f} & 	{_df['mdd'].iloc[1]:.4f} &  {_df['downrisk'].iloc[1]:.4f}   & 	{_df['cumreturn'].iloc[1]:.4f}        \\\ \n" + \
        f"    FPR-0.01             &	{_df['sortino'].iloc[2]:.4f} & {_df['sharpe'].iloc[2]:.4f}	& {_df['risk'].iloc[2]:.4f} & 	{_df['mdd'].iloc[2]:.4f} &  {_df['downrisk'].iloc[2]:.4f}   & 	{_df['cumreturn'].iloc[2]:.4f}        \\\ \n" + \
        f"    FPR-0.05             &  {_df['sortino'].iloc[3]:.4f} & {_df['sharpe'].iloc[3]:.4f}	& {_df['risk'].iloc[3]:.4f} & 	{_df['mdd'].iloc[3]:.4f} &  {_df['downrisk'].iloc[3]:.4f}   & 	{_df['cumreturn'].iloc[3]:.4f}        \\\ \n" + \
        f"    FPR-0.1	          &  {_df['sortino'].iloc[4]:.4f} & {_df['sharpe'].iloc[4]:.4f}	& {_df['risk'].iloc[4]:.4f} & 	{_df['mdd'].iloc[4]:.4f} &  {_df['downrisk'].iloc[4]:.4f}   & 	{_df['cumreturn'].iloc[4]:.4f}        \\\ \n" + \
        f"    FPR-0.2	          &  {_df['sortino'].iloc[5]:.4f} & {_df['sharpe'].iloc[5]:.4f}	& {_df['risk'].iloc[5]:.4f} & 	{_df['mdd'].iloc[5]:.4f} &  {_df['downrisk'].iloc[5]:.4f}   & 	{_df['cumreturn'].iloc[5]:.4f}        \\\ \n" + \
        "    \hline\n" + \
        "\\end{tabular}\n" + \
    "\\end{table}\n"
    print(template)

In [6]:
concat = pd.read_csv("./ts-run-results/[exp2-CSCO]returns的各类指标-cost-汇总.csv")
concat.columns = ["Cost", "Reward", "STR", "SR", "Risk", "MDD", "DR", "CR"]
concat = concat[["Cost", "Reward", "CR", "STR", "SR", "MDD", "Risk", "DR"]]
concat.iloc[:, 2:] = concat.iloc[:, 2:].round(6)
concat.to_csv("./ts-run-results/[exp2-CSCO]returns的各类指标-cost-汇总(换column顺序).csv")

In [47]:
for symbol in symbols:
    convert_latex(symbol, find(symbol))

\begin{table}[H]
    \label{table:CSCO}
    \caption{CSCO}
    \begin{tabular}{l c c c c c c}
    \hline
                     & sortino & sharpe &	risk &  mdd    &  downrisk &	cumreturn     \\ 
    \hline
    B\&H	          &  0.9256 & 0.6606	& 0.2637 &  0.1748 &  0.1882   & 	0.1489        \\ 
    FP5	          &  0.7879 & 0.5576	& 0.2489 & 	0.2032 &  0.1707   & 	0.1131        \\ 
    FPR-0.01             &	0.8582 & 0.5992	& 0.2441 & 	0.2032 &  0.1702   & 	0.1218        \\ 
    FPR-0.05             &  1.1874 & 0.8354	& 0.2214 & 	0.1486 &  0.1535   & 	0.1746        \\ 
    FPR-0.1	          &  2.0383 & 1.3561	& 0.1956 & 	0.1128 &  0.1292   & 	0.2725        \\ 
    FPR-0.2	          &  0.0000 & 0.0000	& 0.0513 & 	0.0601 &  0.0817   & 	0.0000        \\ 
    \hline
\end{tabular}
\end{table}

